In [21]:
from processing.database import Database
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [22]:
db = Database('../data/database.db')
cmt = db.query('SELECT * FROM comments')

In [23]:
print(cmt.info(verbose = True))
print(cmt.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3700001 entries, 0 to 3700000
Data columns (total 29 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   num_sentences          int64 
 1   num_awards             int64 
 2   num_mod_reports        int64 
 3   edited                 int64 
 4   removal_type           int64 
 5   collapsed              int64 
 6   distinguished          int64 
 7   subreddit_type         int64 
 8   author                 object
 9   body                   object
 10  gilded                 int64 
 11  total_awards_received  int64 
 12  created_utc            int64 
 13  author_is_blocked      int64 
 14  archived               int64 
 15  controversiality       int64 
 16  can_mod_post           int64 
 17  id                     object
 18  downs                  int64 
 19  ups                    int64 
 20  link_id                object
 21  locked                 int64 
 22  is_submitter           int64 
 23  parent_

In [24]:
out = cmt.describe().transpose()
out[out['std'] == 0].transpose() # std == 0 means that these are likely useless

,num_awards,num_mod_reports,edited,gilded,total_awards_received,author_is_blocked,can_mod_post,downs
count,3700001.0,3700001.0,3700001.0,3700001.0,3700001.0,3700001.0,3700001.0,3700001.0
mean,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
